# Cоздание аналитической таблицы 



## О данных:

**E-commerce Data - transactions from UK retailer**

Транснациональный набор данных, который содержит все транзакции, произошедшие между 01/12/2010 и 09/12/2011 для зарегистрированного в Великобритании онлайн-магазина розничной торговли.Компания в основном продает уникальные подарки на все случаи жизни. Многие клиенты компании являются оптовиками."

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'C:\Users\123123\Downloads\Pet-projects\datasets\analityc_table.csv', encoding="ISO-8859-1")

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
# заполним пропуски
df = df.fillna(-1)

In [6]:
# переименуем столбцы
df = df.rename(columns={'InvoiceNo':'invoice', 'StockCode':'stock_code', 'Description':'description', 'Quantity':'quantity',
                        'InvoiceDate':'invoice_dt', 'UnitPrice':'price', 'CustomerID':'customer_id', 'Country':'country'})

# приведём дату к типу timestamp
df['invoice_dt'] = pd.to_datetime(df['invoice_dt'], format='%m/%d/%Y %H:%M')

# выделим месяц
df['invoice_month'] = df['invoice_dt'].to_numpy().astype('datetime64[M]')

# посчитаем выручку
df['revenue'] = df['quantity'] * df['price']

### Продажи по месяцам

In [7]:
# сводная таблица продаж по месяцам
revenue_month = df.groupby('invoice_month')['revenue'].agg(['count','sum']).reset_index()
revenue_month

,invoice_month,count,sum
0,2010-12-01,42481,748957.020
1,2011-01-01,35147,560000.260
2,2011-02-01,27707,498062.650
3,2011-03-01,36748,683267.080
4,2011-04-01,29916,493207.121
5,2011-05-01,37030,723333.510
6,2011-06-01,36874,691123.120
7,2011-07-01,39518,681300.111
8,2011-08-01,35284,682680.510
9,2011-09-01,50226,1019687.622


In [8]:
# сформируем диапазон дат по месяцам и запишем в датафрейм
min_dt = df['invoice_month'].min()
max_dt = df['invoice_month'].max()
dt = pd.DataFrame(pd.date_range(min_dt, max_dt, freq='MS'))
dt.columns = ['month']
revenue_month = dt.merge(revenue_month, how='left', left_on='month', right_on='invoice_month')
revenue_month

,month,invoice_month,count,sum
0,2010-12-01,2010-12-01,42481,748957.020
1,2011-01-01,2011-01-01,35147,560000.260
2,2011-02-01,2011-02-01,27707,498062.650
3,2011-03-01,2011-03-01,36748,683267.080
4,2011-04-01,2011-04-01,29916,493207.121
5,2011-05-01,2011-05-01,37030,723333.510
6,2011-06-01,2011-06-01,36874,691123.120
7,2011-07-01,2011-07-01,39518,681300.111
8,2011-08-01,2011-08-01,35284,682680.510
9,2011-09-01,2011-09-01,50226,1019687.622


### Таблица регистрации пользователей

In [9]:
users = df.groupby('customer_id')[['invoice_dt','invoice_month']].agg('min').reset_index()
users.columns = ['customer_id', 'reg_dt', 'reg_month']
users.head()

,customer_id,reg_dt,reg_month
0,-1.0,2010-12-01 11:52:00,2010-12-01
1,12346.0,2011-01-18 10:01:00,2011-01-01
2,12347.0,2010-12-07 14:57:00,2010-12-01
3,12348.0,2010-12-16 19:09:00,2010-12-01
4,12349.0,2011-11-21 09:51:00,2011-11-01


In [10]:
# для объединения dt и users добавим им общий ключ
dt['key'] = 1
users['key'] = 1

# объединим dt u users и удалим ненужный ключ
template = dt.merge(users, on='key').drop('key', axis=1)

# чтобы не хранить ненужные данные отфильтруем месяца и оставим данные только после регистрации
template = template.query('month >= reg_month')
template

,month,customer_id,reg_dt,reg_month
0,2010-12-01,-1.0,2010-12-01 11:52:00,2010-12-01
2,2010-12-01,12347.0,2010-12-07 14:57:00,2010-12-01
3,2010-12-01,12348.0,2010-12-16 19:09:00,2010-12-01
21,2010-12-01,12370.0,2010-12-14 12:58:00,2010-12-01
27,2010-12-01,12377.0,2010-12-20 09:37:00,2010-12-01
...,...,...,...,...
56844,2011-12-01,18280.0,2011-03-07 09:52:00,2011-03-01
56845,2011-12-01,18281.0,2011-06-12 10:53:00,2011-06-01
56846,2011-12-01,18282.0,2011-08-05 13:35:00,2011-08-01
56847,2011-12-01,18283.0,2011-01-06 14:14:00,2011-01-01


### Таблица покупок 

In [11]:
user_month = (df.groupby(['customer_id','invoice_month']).
              agg({'revenue':'sum', 'quantity':'sum', 'description':'nunique'}).
              reset_index())

user_month.columns = ['customer_id', 'month', 'revenue', 'quantity', 'unique_description']
user_month

,customer_id,month,revenue,quantity,unique_description
0,-1.0,2010-12-01,194353.00,45866,2182
1,-1.0,2011-01-01,84925.88,39587,1926
2,-1.0,2011-02-01,61516.50,15156,1602
3,-1.0,2011-03-01,103302.47,7860,1790
4,-1.0,2011-04-01,67159.27,10513,1573
...,...,...,...,...,...
13683,18283.0,2011-10-01,114.65,63,38
13684,18283.0,2011-11-01,651.56,450,116
13685,18283.0,2011-12-01,208.00,142,50
13686,18287.0,2011-05-01,765.28,488,27


### Аналитическая таблица

In [12]:
at = template.merge(user_month, how='left', on=['customer_id', 'month'])
at

,month,customer_id,reg_dt,reg_month,revenue,quantity,unique_description
0,2010-12-01,-1.0,2010-12-01 11:52:00,2010-12-01,194353.00,45866.0,2182.0
1,2010-12-01,12347.0,2010-12-07 14:57:00,2010-12-01,711.79,319.0,31.0
2,2010-12-01,12348.0,2010-12-16 19:09:00,2010-12-01,892.80,1254.0,13.0
3,2010-12-01,12370.0,2010-12-14 12:58:00,2010-12-01,1868.02,968.0,90.0
4,2010-12-01,12377.0,2010-12-20 09:37:00,2010-12-01,1001.52,604.0,43.0
...,...,...,...,...,...,...,...
37444,2011-12-01,18280.0,2011-03-07 09:52:00,2011-03-01,NaN,NaN,NaN
37445,2011-12-01,18281.0,2011-06-12 10:53:00,2011-06-01,NaN,NaN,NaN
37446,2011-12-01,18282.0,2011-08-05 13:35:00,2011-08-01,77.84,28.0,5.0
37447,2011-12-01,18283.0,2011-01-06 14:14:00,2011-01-01,208.00,142.0,50.0


Выполним проверку по сумме выручки

In [13]:
df['revenue'].sum()

9747747.933999998

In [14]:
at['revenue'].sum()

9747747.934

## Метрики на основе аналитической таблицы


### Ежемесячный отчёт по продажам

In [15]:
# определим активных пользователей, заменив  булевы значения - True на 1 пропуски - False на 0
at['active_user'] = (~at['revenue'].isna()) * 1
# определим новых пользователей сравнив месяц покупки с месяцем регистрации
at['cnt_new_users'] = (at['month'] == at['reg_month']) * 1

In [16]:
# помесячный отчёт
report = (at.groupby('month').
        agg({'revenue':'sum', 'customer_id':'nunique', 'active_user':'sum',
             'customer_id':'nunique', 'cnt_new_users':'sum', 'quantity':'sum', 'unique_description':'mean'}).
                 reset_index().rename(columns={'customer_id':'cnt_users'}))

report['avg_quantity_user'] = round(report['quantity'] / report['active_user'])
report['unique_description'] = report['unique_description'].round()
report

,month,revenue,cnt_users,active_user,cnt_new_users,quantity,unique_description,avg_quantity_user
0,2010-12-01,748957.020,949,949,949,342228.0,27.0,361.0
1,2011-01-01,560000.260,1370,784,421,308966.0,28.0,394.0
2,2011-02-01,498062.650,1750,799,380,277989.0,26.0,348.0
3,2011-03-01,683267.080,2190,1021,440,351872.0,27.0,345.0
4,2011-04-01,493207.121,2489,900,299,289098.0,26.0,321.0
5,2011-05-01,723333.510,2768,1080,279,380391.0,26.0,352.0
6,2011-06-01,691123.120,3003,1052,235,341623.0,26.0,325.0
7,2011-07-01,681300.111,3194,994,191,391116.0,27.0,393.0
8,2011-08-01,682680.510,3361,981,167,406199.0,28.0,414.0
9,2011-09-01,1019687.622,3659,1303,298,549817.0,30.0,422.0
